In [ ]:
import os

import cv2
from src.models.AENet import Predictor

IMG_SIZE = 224

In [ ]:
predictor = Predictor()

In [ ]:
base_path = "../../data/celebA-spoof/CelebA_Spoof_/CelebA_Spoof/Data/test/"
live_paths = [
    os.path.join(base_path, "4930/live/504311.png"),
    os.path.join(base_path, "4930/live/511717.png"),
    os.path.join(base_path, "4931/live/494918.png"),
    os.path.join(base_path, "4943/live/494437.png"),
    os.path.join(base_path, "4973/live/494955.png"),
]
spoof_paths = [
    os.path.join(base_path, "4930/spoof/495600.png"),
    os.path.join(base_path, "3613/spoof/541354.png"),
    os.path.join(base_path, "4931/spoof/497474.png"),
    os.path.join(base_path, "4943/spoof/494824.png"),
    os.path.join(base_path, "4973/spoof/502204.png"),
]
bbox_lives = ["".join(path.split(".")[:-1]) + "_BB.txt" for path in live_paths]
bbox_spoofs = ["".join(path.split(".")[:-1]) + "_BB.txt" for path in spoof_paths]

print(live_paths)
print(spoof_paths)
print(bbox_lives)
print(bbox_spoofs)

In [ ]:
live_images = []
spoof_images = []
bb_lives = []
bb_spoofs = []

for live_path, spoof_path, bb_live, bb_spoof in zip(live_paths, spoof_paths, bbox_lives, bbox_spoofs):
    # Load the images
    live_images.append(cv2.imread(live_path))
    spoof_images.append(cv2.imread(spoof_path))

    # Extract bounding boxes
    with open(bb_live, "r") as file:
        bb_lives.append(list(map(int, file.readlines()[0].split()[:-1])))

    with open(bb_spoof, "r") as file:
        bb_spoofs.append(list(map(int, file.readlines()[0].split()[:-1])))

print(live_images[0].shape)
print(spoof_images[0].shape)
print(bb_lives[0])
print(bb_spoofs[0])
bbox_lives = bb_lives
bbox_spoofs = bb_spoofs

In [ ]:
for i in range(len(live_images)):
    # Resize images
    live_images[i] = cv2.resize(live_images[i], (IMG_SIZE, IMG_SIZE))
    spoof_images[i] = cv2.resize(spoof_images[i], (IMG_SIZE, IMG_SIZE))

    # Crop faces and show cropped images
    bbox_live = bbox_lives[i]
    bbox_spoof = bbox_spoofs[i]

    live_images[i] = live_images[i][
        bbox_live[1] : bbox_live[1] + bbox_live[3],
        bbox_live[0] : bbox_live[0] + bbox_live[2],
    ]
    spoof_images[i] = spoof_images[i][
        bbox_spoof[1] : bbox_spoof[1] + bbox_spoof[3],
        bbox_spoof[0] : bbox_spoof[0] + bbox_spoof[2],
    ]

In [ ]:

for live in live_images:
    rst1 = predictor.predict([live])[0]
    print(rst1[1])
for spoof in spoof_images:
    rst2 = predictor.predict([spoof])[0]
    print(rst2[1])